# Structured Output mit LLMs in LangChain

In diesem Notebook lernen Sie, wie Sie mit LangChain strukturierte Ausgaben von LLMs erzeugen können. Strukturierte Ausgaben sind wichtig für die programmatische Weiterverarbeitung in Anwendungen.

## 1. Grundlagen

Zunächst importieren wir die nötigen Bibliotheken und stellen sicher, dass wir ein LLM zur Verfügung haben.

In [4]:
from helpers import llm
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# Test des LLM
response = llm().invoke("Was ist eine strukturierte Ausgabe?")
print(response.content)

Eine strukturierte Ausgabe bezieht sich auf die Präsentation von Informationen in einem organisierten und klaren Format, das leicht verständlich und interpretierbar ist. Diese Art der Ausgabe ist besonders nützlich, um Daten oder Informationen so darzustellen, dass sie effizient analysiert, verglichen oder weiterverarbeitet werden können. 

Beispiele für strukturierte Ausgaben sind:

1. **Tabellen**: Daten werden in Zeilen und Spalten organisiert, was den Vergleich verschiedener Datensätze erleichtert.
   
2. **Listen**: Informationen werden in geordneter oder ungeordneter Form präsentiert, um bestimmte Punkte klar hervorzuheben.
   
3. **Diagramme und Grafiken**: Visuelle Darstellungen von Daten, die Muster oder Trends verdeutlichen können.
   
4. **Berichte**: Dokumente, die Informationen in bestimmten Abschnitten und Unterabschnitten strukturiert darstellen, oft mit Überschriften, Unterüberschriften und nummerierten Absätzen.
   
5. **XML/JSON**: Formatierte Datenstrukturen, die häu

## 2. Einfache strukturierte Ausgabe durch Anweisungen

Der einfachste Weg, strukturierte Ausgaben zu erhalten, ist durch klare Anweisungen im Prompt.

In [7]:
# Einfaches Prompt-Template für strukturierte Ausgabe
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfsbereicher Assistent, der Daten im reinen JSON-Format zurückgibt."),
    ("human", "Gib mir Informationen über die Stadt {stadt} im JSON-Format mit den Feldern 'name', 'land', 'einwohner' und einem Array 'sehenswuerdigkeiten'.")
])

# Chain erstellen
chain = prompt | llm(temperature=0.1) | StrOutputParser()

# Chain ausführen
result = chain.invoke({"stadt": "Berlin"})
print(result)

```json
{
  "name": "Berlin",
  "land": "Deutschland",
  "einwohner": 3645000,
  "sehenswuerdigkeiten": [
    "Brandenburger Tor",
    "Reichstagsgebäude",
    "Berliner Mauer",
    "Fernsehturm",
    "Museumsinsel",
    "East Side Gallery",
    "Checkpoint Charlie",
    "Berliner Dom",
    "Schloss Charlottenburg",
    "Gedenkstätte Berliner Mauer"
  ]
}
```


Das funktioniert, aber die Ausgabe ist nur ein String im JSON-Format. Wir können das Ergebnis manuell in ein Python-Objekt umwandeln:

In [8]:
import json

# String in JSON-Objekt umwandeln
try:
    data = json.loads(result)
    print(f"Name: {data['name']}")
    print(f"Land: {data['land']}")
    print(f"Einwohner: {data['einwohner']}")
    print(f"Sehenswürdigkeiten: {', '.join(data['sehenswuerdigkeiten'])}")
except json.JSONDecodeError as e:
    print(f"Fehler beim JSON-Parsing: {e}")
    print("Rohausgabe:")
    print(result)

Fehler beim JSON-Parsing: Expecting value: line 1 column 1 (char 0)
Rohausgabe:
```json
{
  "name": "Berlin",
  "land": "Deutschland",
  "einwohner": 3645000,
  "sehenswuerdigkeiten": [
    "Brandenburger Tor",
    "Reichstagsgebäude",
    "Berliner Mauer",
    "Fernsehturm",
    "Museumsinsel",
    "East Side Gallery",
    "Checkpoint Charlie",
    "Berliner Dom",
    "Schloss Charlottenburg",
    "Gedenkstätte Berliner Mauer"
  ]
}
```


## 3. Strukturierte Ausgabe mit Pydantic-Modellen

Ein zuverlässigerer Ansatz ist die Verwendung von Pydantic-Modellen zur Definition der erwarteten Struktur.

In [9]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definition des Ausgabeformats als Pydantic-Modell
class Stadt(BaseModel):
    name: str = Field(description="Der Name der Stadt")
    land: str = Field(description="Das Land, in dem die Stadt liegt")
    einwohner: int = Field(description="Die Anzahl der Einwohner")
    sehenswuerdigkeiten: List[str] = Field(description="Liste bekannter Sehenswürdigkeiten")

# Parser erstellen
parser = PydanticOutputParser(pydantic_object=Stadt)

# Prompt mit Parser-Anweisungen
# Um in dem Anweisungstext aus der funktion get_format_instruction die felder zu escapen
# ersetzten wir die { mit {{ und } mit }}. 
# Damit versucht die Template Engine nicht die Anweisung als properties auszuwerten
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfsbereicher Assistent, der Informationen in strukturierter Form zurückgibt."),
    ("human", "Gib mir Informationen über die Stadt {stadt} "+parser.get_format_instructions().replace("}","}}").replace("{","{{")),
])

# Chain erstellen
chain = prompt | llm(temperature=0.1) | parser

# Chain ausführen
result = chain.invoke({"stadt": "München"})

# Auf strukturierte Daten zugreifen
print(f"Name: {result.name}")
print(f"Land: {result.land}")
print(f"Einwohner: {result.einwohner}")
print(f"Sehenswürdigkeiten: {', '.join(result.sehenswuerdigkeiten)}")

KeyError: 'Input to ChatPromptTemplate is missing variables {\'"properties"\', \'"foo"\'}.  Expected: [\'"foo"\', \'"properties"\', \'stadt\'] Received: [\'stadt\']\nNote: if you intended {"properties"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"properties"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT'

## 4. Anwendungsbeispiel: Sentiment-Analyse mit strukturierter Ausgabe

Ein praktisches Beispiel für strukturierte Ausgaben ist die Sentiment-Analyse von Kundenfeedback. Wir verwenden einen vordefinierten Prompt aus dem LangChain Hub.

In [11]:
from langchain import hub

# Prompt aus dem Hub laden
sentiment_prompt = hub.pull("borislove/customer-sentiment-analysis")

# Beispiel-Kundenbrief
client_letter = """Ich bin von dem Volleyballschläger zutiefst enttäuscht. Zuerst ist der Griff abgefallen, danach auch noch der Dynamo. Außerdem riecht er noch schlechter als er schmeckt. Wieso ist das immer so ein Ärger mit euch?"""

# Standard-Format-Anweisungen
format_instructions = """Klassifiziere den Kundenbrief nach Stimmung von 1-5, wobei 1 sehr negativ und 5 sehr positiv ist."""

# Chain erstellen
sentiment_chain = sentiment_prompt | llm(temperature=0.1) | StrOutputParser()

# Chain ausführen
result = sentiment_chain.invoke({"client_letter": client_letter, "format_instructions": format_instructions})
print(result)

Unknown


## 5. Erweiterte strukturierte Ausgabe für die Sentiment-Analyse

Nun passen wir die Format-Anweisungen an, um eine detailliertere strukturierte Ausgabe zu erhalten.

In [12]:
# Erweiterte Format-Anweisungen
format_instructions = """Zusätzlich zur numerischen Klassifizierung (1-5) sollst du:
1. Die konkreten Kritikpunkte in Stichpunkten zusammenfassen
2. Einen Vorschlag machen, was dem Kunden geantwortet werden sollte
3. Drei konkrete Maßnahmen zur Produktverbesserung empfehlen

Formatiere die Ausgabe im folgenden JSON-Format ohne codeblock:
{
    "bewertung": Zahl zwischen 1-5,
    "kritikpunkte": ["Liste der Kritikpunkte"],
    "antwortvorschlag": "Vorschlag für Antwort an den Kunden",
    "verbesserungsvorschlaege": ["Liste mit Verbesserungsvorschlägen"]
}
"""

# Chain ausführen
result = sentiment_chain.invoke({"client_letter": client_letter, "format_instructions": format_instructions})


# JSON parsen (wenn die Ausgabe korrekt formatiert ist)
try:
    data = json.loads(result)
    print("\nStrukturierte Daten:")
    print(f"Bewertung: {data['bewertung']}")
    print("Kritikpunkte:")
    for punkt in data['kritikpunkte']:
        print(f"- {punkt}")
    print(f"\nAntwortvorschlag: {data['antwortvorschlag']}")
    print("\nVerbesserungsvorschläge:")
    for vorschlag in data['verbesserungsvorschlaege']:
        print(f"- {vorschlag}")
except Exception as e:
    print(f"Fehler beim Parsen: {e}")

```json
{
    "bewertung": 3,
    "kritikpunkte": [
        "Griff des Volleyballschlägers ist abgefallen",
        "Dynamo des Volleyballschlägers ist abgefallen",
        "Der Volleyballschläger riecht unangenehm"
    ],
    "antwortvorschlag": "Sehr geehrter Kunde, es tut uns leid zu hören, dass Sie mit dem Volleyballschläger unzufrieden sind. Wir möchten Ihnen gerne helfen, dieses Problem zu lösen. Bitte senden Sie uns den Schläger zurück, damit wir ihn überprüfen und Ihnen entweder einen Ersatz oder eine Rückerstattung anbieten können. Vielen Dank für Ihr Verständnis und Ihre Geduld.",
    "verbesserungsvorschlaege": [
        "Qualitätskontrolle der Griffe verbessern, um sicherzustellen, dass sie fest angebracht sind",
        "Überprüfung und Verbesserung der Befestigung des Dynamos",
        "Materialien verwenden, die keinen unangenehmen Geruch abgeben"
    ]
}
```
Fehler beim Parsen: Expecting value: line 1 column 1 (char 0)


## 6. Übungsaufgabe: Produktbewertungsanalyse

Erstellen Sie ein Prompt-Template, das Kundenbewertungen analysiert und in eine strukturierte Form bringt.

In [ ]:
# Prompt-Template für Produktbewertungsanalyse
product_review_prompt = ChatPromptTemplate.from_messages([
    ("system", """Du bist ein Experte für die Analyse von Produktbewertungen. 
    Extrahiere Informationen aus Kundenbewertungen und gib sie im folgenden JSON-Format ohne codeblock zurück:
    {{
        "produktname": "Name des Produkts",
        "gesamtbewertung": Zahl zwischen 1-5,
        "positive_punkte": ["Liste positiver Aspekte"],
        "negative_punkte": ["Liste negativer Aspekte"],
        "verbesserungsvorschlaege": ["Liste von Verbesserungsvorschlägen"]
    }}
    """
    ),
    ("human", "Analysiere folgende Produktbewertung: {review}"),
])

# Beispielbewertung
review = """Der Kaffeevollautomat XYZ-5000 hat mich größtenteils überzeugt. 
Die Bedienung ist super einfach und intuitiv, das Design passt gut in meine Küche. 
Der Kaffee schmeckt ausgezeichnet und hat eine gute Crema. 
Allerdings ist die Maschine sehr laut beim Mahlen und die Reinigung des Milchsystems ist umständlich. 
Die App-Steuerung stürzt manchmal ab. Ich würde mir ein leiseres Mahlwerk und ein einfacheres Reinigungssystem wünschen."""

# Chain erstellen und ausführen
review_chain = product_review_prompt | llm(temperature=0.1) | StrOutputParser()
result = review_chain.invoke({"review": review})

# Strukturierte Daten extrahieren
try:
    data = json.loads(result)
    print("\nAnalyse der Produktbewertung:")
    print(f"Produkt: {data['produktname']}")
    print(f"Gesamtbewertung: {data['gesamtbewertung']}/5")
    
    print("\nPositive Aspekte:")
    for punkt in data['positive_punkte']:
        print(f"✓ {punkt}")
        
    print("\nNegative Aspekte:")
    for punkt in data['negative_punkte']:
        print(f"✗ {punkt}")
        
    print("\nVerbesserungsvorschläge:")
    for vorschlag in data['verbesserungsvorschlaege']:
        print(f"→ {vorschlag}")
except Exception as e:
    print(f"Fehler beim Parsen: {e}")

## 7. Tipps für effektive strukturierte Ausgaben

1. **Klare Anweisungen**: Je präziser die Anweisungen, desto konsistenter die Ausgaben
2. **Beispiele**: Geben Sie Beispiele für das gewünschte Format
3. **Niedrige Temperatur**: Verwenden Sie eine niedrige Temperatur (0-0.2) für konsistentere Ergebnisse
4. **Fehlerbehandlung**: Implementieren Sie Validierung und Fehlerbehandlung
5. **Fallback-Mechanismen**: Bereiten Sie Ausweichlösungen vor, falls das LLM nicht die erwartete Struktur liefert

## 8. Zusammenfassung

In diesem Notebook haben wir gelernt:
- Wie man durch einfache Anweisungen strukturierte Textausgaben erzeugt
- Wie man mit Pydantic-Modellen typsichere strukturierte Ausgaben erhält
- Wie man vordefinierte Prompts aus dem LangChain Hub anpasst
- Praktische Anwendungen für strukturierte Ausgaben wie Sentiment-Analyse und Produktbewertungen

Diese Techniken sind essenziell für die Entwicklung robuster KI-Anwendungen, die zuverlässig mit den generierten Daten arbeiten müssen.